## Exploratory Data Analysis With Python
### What is Exploratory Analysis?
A preliminary step in data analysis to: 

1. Summarize main characteristics of the data
2. Gain better understanding of the data
3. Uncover relationships between the data set
4. Extract important variables  

Our goal here being to answer the question:  
"**What are the characteristics that have the most impact on the car price**?"

---

### Introduction

Using a test notebook, we start with an import of a few essential libraries  
and assigning the data we want to use to a var.  

Then we want to initiate our data frame! 

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats as sts

df_data = Path().cwd().parent.parent/"Data"/"Clean_Data"/"clean_auto_df.csv"
auto_df = pd.read_csv(df_data)

We've already wrangled the data we're using, it's time to focus on what the  
data could be telling us. We'll work on visualizing the data to observe  
association.

**Functions commonly used in EDA:**  

- describe()
- info()
- value_counts()
- linspace()
- cut()
- boxplot()
- scatter()
- show()
- groupby()
- get_group()
- f_oneway()
- corr()  

---  

### Descriptive Statistics  

Below, we start with descriptive statistics, which tells us a summary of the  
basic statistical numerical data, excluding object and bool types unless  
passed a paramater to do so. Naan values are also excluded in these statistics.  

**Ex.**  
`print(auto_df.describe(include='all'))` *A special string keyword in pandas*  

`print(auto_df.describe(include=[bool]))` *Only describes columns with bool*  

This function will give you a clearer idea of the distribution of your  
different variables; later we'll visualize this data with a boxplot.  

In [ ]:
# auto_df.head(20)
print(auto_df.describe())

**What about the descriptive statistics for our categorical data?**  

To get quick descriptive information in a user-friendly output, you want to  
pair `value_counts()` with `to_frame()`. Value_counts() to get the frequency  
of each value in the specified column--for example, `auto_df["drive-wheels"]`,  
and then to_frame() for the desired presentation.      

In [ ]:
drive_wheel_counts = auto_df["drive-wheels"].value_counts().to_frame()
drive_wheel_counts.columns = ['value_counts']
drive_wheel_counts.index.name = 'drive-wheels'

print(drive_wheel_counts)

The next important part of EDA and descriptive analytics is visualization.  

We can visualize data with one of the most common EDA tools which allows us to  
determine data distribution at a glance.  

It is *primarily effective* with **grouped categorical data** (for example,  
drive-wheels) and sometimes binned continuous/numerical data. With  
**categorical data** we can see how our target variable compares to our  
predictor variable. For **numerical data** we can analyze a column with a wide  
range to compare your target variable across those ranges (low, med, high).  

**What boxplots can tell you**  

- **Box (IQR):** The interquartile range (Q1 to Q3) shows where the middle 50%  
  of your data lies. A small box means low variability; a large box means  
  greater spread among typical values.

- **Median line:** This tells you the central value. If it’s not centered in  
  the box, the distribution is skewed (e.g., median closer to  
  Q1 = positive/right skew).

- **Whiskers:** Typically extend to 1.5×IQR or to min/max in your data. If  
  whiskers are uneven in length, this hints at skewness or asymmetry.

- **Outliers (dots):** Individual points beyond the whiskers flag unusual  
  values. A high number of outliers might suggest:

  - *Heavy-tailed distribution*

  - *Data entry errors*
  
  - *Real-world phenomena needing explanation*  

If there is **significant overlap** in the IQR of different groups, it is NOT  
a good predictor of price.  

In [ ]:
sns.boxplot(y="price", x="drive-wheels", data=auto_df)

In [ ]:
# Here is an example using binning on numerical data
bins = np.linspace(min(auto_df["horsepower"]), max(auto_df["horsepower"]), 4)
group_names = ['low', 'medium', 'high']

hp_binned = pd.cut(
    auto_df["horsepower"],
    bins,
    labels=group_names,
    include_lowest=True
)
hp_df = hp_binned.to_frame(name="horsepower-binned")
hp_df["price"] = auto_df["price"]

# hp_df.head()


sns.boxplot(y="price", x="horsepower-binned", data=hp_df)

In [ ]:
plt.hist(auto_df["horsepower"], bins=10)
plt.xlabel("Horsepower")
plt.ylabel("Frequency")
plt.title("Horsepower Distribution")
plt.show()


Another visualization tool we can use is the **scatter plot**. Truly made for  
comparing continuous data (target and predictor), it explores the relationship  
between the two to identify patterns or trends.

In [ ]:
y = auto_df["price"]
x = auto_df["engine-size"]
plt.scatter(x,y)

plt.title("Scatterplot of Engine Size vs Price")
plt.ylabel("Price")
plt.xlabel("Engine Size")
plt.show

---

### Groupby()  

While you can apply groupby() to numerical data, it works best when there are  
multiple of the same unique values in categorical data. It helps explore  
relationships between your target variable and predictors, and if different  
categories or category pairs explain variation in the target.  

`groupby()` method: 

- Can be applied on categorical variables  
- Group Data into categories  
- Single or multiple variables

In [ ]:
# Finding the average price of vehicles and observing how they differ between
# categories of "body-style" and "drive-wheels".

df_test = auto_df[['drive-wheels', 'body-style', 'price']]

# Grouping by just one category
df_group = df_test.groupby(['drive-wheels'], as_index=False)["price"].mean()
print(df_group)

# as_index=False means the group keys (drive-wheels, body-style) will not 
# become the index of the result--best for visualization.
df_group2 = df_test.groupby(
    ['drive-wheels',
    'body-style'],
    as_index=False
).mean()

print(df_group2)

In [ ]:
# Best for multi-level indexing and easy slicing
index_test = auto_df[['drive-wheels', 'body-style', 'price']]
index_group = index_test.groupby(['drive-wheels', 'body-style']).mean()

avg_rwd_convertible = index_group.loc[("rwd", "convertible")]
print(avg_rwd_convertible)

---  

### Pivot() Method  

While the output of our groupby methods is suitable for some cases, you may  
find yourself wanting something easier to read when moving from a data frame  
(long) structure to a wider layout.  

Enter the `pivot()` method.

Here, one vsriable is displayed along the columns and the other variable is  
displayed along the rows (as opposed to fatures being columns).  



In [ ]:
df_pivot = df_group2.pivot(index="drive-wheels", columns="body-style")
df_pivot

Once we have the pivot table, a great way to view the target variable across  
multiple variables for visual clues about relationship is with a **heatmap**.  

In [ ]:
sns.heatmap(df_pivot, cmap='RdBu')
plt.ylabel("drive-wheels")
plt.xlabel("body-style")
plt.show()

---  

### Analysis of Variance (ANOVA)

ANOVA is a statistical method used to compare the means of two or more groups  
to determine if there is a significant difference between them. It answers  
questions like:  

- Do different car brands have significantly different average prices?
- Does a new drug have a different effect than existing treatments?  

ANOVA returns two values:  
1. **F-static score (F-score):** Measures how much variation exists *between*  
group means compared to the variation within each group  
(F=variation between group means/variation within groups)
   - High f-score means the group means are significantly different
   - Low f-score means the group means are very similar  

1. **P-value:** Determines if the difference is statistically significant  
   - A p-value < 0.05 suggests a strong difference (reject null hypothesis)
   - A p-value > 0.05 suggests there is no strong difference (failt to reject  
    null hypothesis)  



In [ ]:
df_anova = auto_df[["make", "price"]]
anova_grouped = df_anova.groupby("make")

anova_results = sts.f_oneway(
    anova_grouped.get_group("honda")["price"],
    anova_grouped.get_group("subaru")["price"]
)

# F-score low, p-val > 0.05. No meaningful price difference between 
# these brands
print(anova_results)

In [ ]:
# What about testing your entire category/independent variable at one time?
# For that, we use Boolean indexing

# Look in grouped_test, keep rows where drive-wheels is 'x' value (our 
# modular condition), and give me the corresponding price from the "price"
# column
grouped_test = auto_df[["drive-wheels", "price"]]
group1 = grouped_test.loc[grouped_test["drive-wheels"] == "fwd", "price"]
group2 = grouped_test.loc[grouped_test["drive-wheels"] == "rwd", "price"]
group3 = grouped_test.loc[grouped_test["drive-wheels"] == "4wd", "price"]

anova_results2 = sts.f_oneway(group1, group2, group3)
print(f"The ANOVA is: {anova_results2}")

# F-score high, p-val extremely small. There is a strong, meaningful difference
# between drive-wheels

---  

### Correlation  

Correlation is the statistical metric for measuring to what extent different  
numerical variables are interdependent. In other words, when we look at  
two variables time, if one variable changes, how does this effect change  
in the other variable?

We view this correlation on a scatter plot with an included regression line:  
- Steep inclines indicate a strong predictor, flat indicates poor
- Upward slope = positive correlation, downward slope = negative correlation  

*Note: Correlation is not causation. DA mostly deals with correlation.*

In [ ]:
sns.regplot(x="horsepower", y="price", data=auto_df)
plt.ylim(0,)
plt.show()

### Correlation Statistics (`stats.pearsonr()`) 

**Pearson Correlation** measures the strength of the correlation between two  
features. It returns two values:
- Correlation coefficient
  - **Close to +1**: Large positive relationship  
  - **Close to -1**: Large negative relationship  
  - **Close to 0**: No relationship  
- P-value
  - **P-value < 0.001**: Strong certainty in the result  
  - **p-value < 0.05**: Moderate certainty in the result  
  - **P-value < 0.1**: Weak certainty in the result  
  - **P-value > 0.1**: NO certainty in the result  

For the sake of our future modeling, we would go through each numerical feature  
and determine the correlation with price to identify the strongest influencing  
features.

In [ ]:
pearson_coef, p_value = sts.pearsonr(auto_df["horsepower"], auto_df["price"])
print(f"The Pearson Coef: {pearson_coef}")
print(f"P-value: {p_value}")

### Correlation Heatmap  

An extremely quick way to both process the pearson coefficient for each data  
category compared to every other category. In this instance, we can see how  
every other category stacks with our target variable ("price").  

We must first correlate all of the categories in the df with Pandas' built in  
method, `corr()`. Once calculated, we can use `seaborn's heatmap()` function to  
visualize the distribution.

In [ ]:
corr = auto_df.corr(numeric_only=True)

h_map = sns.heatmap(corr, cmap='coolwarm')
plt.show()

# Exploring correlation between specific features
print(auto_df[["engine-size", "price"]].corr())